In [4]:
import pandas as pd
import re
import requests
import json


In [32]:
def get_blocks():
    sats_conversion = 100000000
    last_blocks = requests.get('https://explorer.rise.vision/api/getLastBlocks')
    last_blocks_cleaned = json.loads(last_blocks.content)['blocks']
    df = pd.DataFrame(last_blocks_cleaned)
    lst = []
    for dct in df['delegate'].tolist():
        lst.append((dct['username'], dct['rank']))

    df['username'] = ''
    df['rank'] = ''

    for ix in range(len(df)):
        df['username'].ix[ix] = lst[ix][0]
        df['rank'].ix[ix] = lst[ix][1]

    df.head(5)
    del df['delegate']

    df_cols = [x for x in df.columns.tolist()]
    block_cols = ['generator', 'height', 'blockid', 'reward', 'timestamp', 'totalamount', 'totalfee',
                  'totalforged', 'transactionscount', 'username', 'rank']
    columns = {x: y for (x, y) in zip(df_cols, block_cols)}
    df.rename(columns=columns, inplace=True)

    df['reward'] = df['reward']/sats_conversion
    df['totalamount'] = df['totalamount']/sats_conversion
    df['totalfee'] = df['totalfee']/sats_conversion
    df['totalforged'] = df['totalforged']/sats_conversion

    return df

df = get_blocks()

/home/anand/Desktop/mysite/venv/lib/python2.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/home/anand/Desktop/mysite/venv/lib/python2.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app


In [33]:
df

,generator,height,blockid,reward,timestamp,totalamount,totalfee,totalforged,transactionscount,username,rank
0,13092524449561764982R,1431966,17455020386728998073,12.0,76737990,0.000000,0.0,12.0,0,zen,20
1,16644795871073778224R,1431965,6624182396050888577,12.0,76737960,0.000000,0.0,12.0,0,coinbang,54
2,2780996313277825383R,1431964,8238212902744576371,12.0,76737930,0.000000,0.0,12.0,0,cct_risearmyx,7
3,3511748430431975976R,1431963,16261582681233475090,12.0,76737900,0.000000,0.0,12.0,0,viking_pool,5
4,572850168092752288R,1431962,14458214515075826176,12.0,76737870,0.000000,0.0,12.0,0,indianajones,101
5,15832203486816837180R,1431961,15498136016240561568,12.0,76737840,0.000000,0.0,12.0,0,goldengate,61
6,5465185310874283336R,1431960,3396856107355393582,12.0,76737810,0.000000,0.0,12.0,0,gregorst,4
7,8093718274007724701R,1431959,13197478803091275435,12.0,76737780,183.078836,0.9,12.9,9,official_pool,1
8,1446929241876228326R,1431958,10782221333247578313,12.0,76737750,28.559014,0.6,12.6,6,risearmyxx,10
9,2797468989083501185R,1431957,6350867268376240013,12.0,76737720,33.236435,1.2,13.2,12,tiesto,100


In [34]:
def get_transactions():
    sats_conversion = 100000000
    last_transactions = requests.get('https://explorer.rise.vision/api/getLastTransactions')
    last_transactions_cleaned = json.loads(last_transactions.content)['transactions']
    zf = pd.DataFrame(last_transactions_cleaned)

    to_del = ['asset','blockId','knownRecipient','knownSender','recipientDelegate','recipientPublicKey','requesterPublicKey','rowId','senderDelegate','senderPublicKey','signSignature','signature','signatures',]
    desired_cols = [x for x in zf.columns.tolist() if x not in to_del]

    zf = zf[desired_cols]

    zf = zf.rename(columns={x[0]:x[1].lower() for x in zip(zf.columns.tolist(),zf.columns.tolist())})

    zf = zf[['id','amount','fee','height','senderid','timestamp','recipientid','type','confirmations']]

    sort_me = """
    txid = models.BigIntegerField()
    amount = models.BigIntegerField()
    fee = models.BigIntegerField()
    height = models.BigIntegerField()
    senderid = models.CharField(max_length=50)
    timestamp = models.BigIntegerField()
    recipientid = models.CharField(max_length=50)
    txtype = models.BigIntegerField()
    confirmations = models.BigIntegerField()
    """

    lst = sort_me.split('\n')
    tx_cols = []
    for ix in lst:
        if ix:  
            resp = re.split(' =', ix)
            tx_cols.append(resp[0].strip())
    curr_tx_cols = zf.columns.tolist()
    columns = {x:y for (x,y) in zip(curr_tx_cols, tx_cols)}
    zf.rename(columns = columns, inplace=True)
                           
    zf['amount'] = zf['amount']/sats_conversion
    zf['fee'] = zf['fee']/sats_conversion

    return zf
zf = get_transactions()

In [35]:
zf

,txid,amount,fee,height,senderid,timestamp,recipientid,txtype,confirmations
0,594983672981728420,50.937729,0.1,1431959,14056190751918729107R,76737763,937582390950725760R,0,8
1,7879793490752863749,36.110544,0.1,1431959,14056190751918729107R,76737760,18128714322464507853R,0,8
2,7654111836611777455,26.625777,0.1,1431959,14056190751918729107R,76737756,8114325576201566754R,0,8
3,2825821505357475986,25.882729,0.1,1431959,14056190751918729107R,76737753,12859719986607278038R,0,8
4,10673384279291551556,10.626215,0.1,1431959,14056190751918729107R,76737750,12182186558361863135R,0,8
5,13277158371235132311,10.211187,0.1,1431959,14056190751918729107R,76737746,8506594824957938347R,0,8
6,13734879668783433827,7.932184,0.1,1431959,14056190751918729107R,76737743,880588348496719012R,0,8
7,173128647292465511,7.393279,0.1,1431959,14056190751918729107R,76737739,8866426602287223413R,0,8
8,3411709518379710721,7.359191,0.1,1431959,14056190751918729107R,76737736,6590748823095783055R,0,8
9,18078787765827267394,5.509688,0.1,1431958,14056190751918729107R,76737733,1895332526999142055R,0,9
